In [1]:
! sudo apt-get install mongodb
! sudo apt-get update
! sudo service mongodb start

Reading package lists... Done
Building dependency tree       
Reading state information... Done
mongodb is already the newest version (1:3.6.3-0ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu 

In [2]:
!pip install fasttext

In [3]:
from pymongo import MongoClient

In [2]:
from pprint import pprint

In [4]:
# Conexion con el servidor
client = MongoClient('localhost')

In [5]:
# Definiendo la base de datos
db = client['ABSAReseñas']

In [6]:
# Definiendo colecciones
trainigdb = db['TrainingSet']
'''testdb = db['TestSet']
trialdb = db['TrialSet']'''

"testdb = db['TestSet']\ntrialdb = db['TrialSet']"

In [7]:
! mongoimport -d 'ABSAReseñas' -c 'TrainingSet' --type json --file TestSet.json

2020-10-22T21:20:59.939+0000	connected to: localhost
2020-10-22T21:20:59.990+0000	imported 268 documents


In [8]:
import fasttext.util
import numpy as np

In [9]:
fasttext.util.download_model('es', if_exists='ignore')

'cc.es.300.bin'

In [10]:
ft = fasttext.load_model('cc.es.300.bin')

In [12]:
for doc in trainigdb.find({}):
  sentence_vec = []
  vec = 0
  for sentence in doc['Sentences']:
    vec = vec + ft.get_sentence_vector(sentence)/len(sentence)
    sentence_vec.append((ft.get_sentence_vector(sentence)).tostring())
    
  trainigdb.update_one({'ID':doc['ID']},{'$set':{'Review_Vec':vec.tostring(),'Sentences_Vec':sentence_vec}})



In [13]:
for doc in trainigdb.find({}).limit(5):
  pprint(doc)

{'_id': ObjectId('5f8fb73d7f67a5fb46b55090'), 'ID': 'es_9reinas_0_MonicaNavarroOlive_2015-04-19', 'Review': 'La comida estuvo muy sabrosa. Quien sea amante de la carne tiene una carta bastante amplia para elegir., aunque ayer  no tenían  chuleton. Lo único que nos sorprendió es que nos sirvieran los entrantes y los platos principales a la vez.', 'Sentences': ['La comida estuvo muy sabrosa.', 'Quien sea amante de la carne tiene una carta bastante amplia para elegir., aunque ayer  no tenían  chuleton.', 'Lo único que nos sorprendió es que nos sirvieran los entrantes y los platos principales a la vez.'], 'Aspects': ['FOOD', 'FOOD', 'SERVICE', 'RESTAURANT'], 'Polarities': ['positive', 'positive', 'negative', 'positive'], 'CleanReview': 'La comida estuvo muy sabrosa. Quien sea amante de la carne tiene una carta bastante amplia para elegir., aunque ayer  no tenían  chuleton. Lo único que nos sorprendió es que nos sirvieran los entrantes y los platos principales a la vez.', 'CleanSentences': 

In [14]:
for doc in trainigdb.find({}).limit(1):
  print('Vector de Reseña: ')
  x =np.frombuffer(doc['Review_Vec'],dtype='float32')
  print(x)
  for sentence in doc['Sentences_Vec']:
    print('Vector en Enunciados: ')
    print(np.frombuffer(sentence,dtype='float32'))

Vector de Reseña: 
[ 2.00648280e-03 -2.44322652e-03  6.61147118e-04  2.92635290e-04
 -6.92174217e-06 -2.89390329e-04  6.72549126e-04 -3.97276919e-04
 -4.58879658e-04 -4.18741396e-03 -4.68917890e-04 -1.12248573e-03
  1.52892942e-04  1.29969430e-03  6.63815998e-04  2.77012907e-04
  1.10007485e-03 -3.54163058e-04  1.24688391e-04  9.25174099e-04
 -1.89997896e-03 -1.68697396e-03  5.85372210e-04  5.58456872e-04
  4.63834935e-04 -5.05527714e-03 -3.33804288e-04 -2.28656005e-04
 -6.81947742e-04  1.60022185e-03  1.36130431e-03 -2.90385098e-04
  4.47320665e-04  5.95166639e-04 -1.89091859e-03  1.33132446e-04
 -4.54617111e-04  1.59898133e-04  2.52062804e-04  1.13391157e-04
 -3.40810092e-03 -1.11714716e-03 -1.79142843e-03  3.51980678e-04
 -2.74151680e-03 -4.09236411e-04 -1.87330996e-03  6.59235171e-04
 -5.12158294e-06 -1.48084131e-03  1.08887826e-03  3.02660884e-03
  2.13404698e-03 -6.82627957e-04  6.37092919e-04 -2.69859098e-04
 -2.80002668e-03  1.51327823e-03  2.84397047e-05 -1.42427208e-03
 -1.34

# Jorge, NO toques lo de abajo

In [ ]:
 '''
 x = ft.get_word_vector('uno')
 xs = x.tostring()
 xx = np.frombuffer(xs,dtype='float32')
 '''